In [1]:
cd /data/projects/windturbine-identification/MachineLearningCourse

/data/projects/windturbine-identification/MachineLearningCourse


In [56]:
"""
Created on Mon Feb 18 20:30:48 2019

@author: jschmidt
"""

import os, shutil, random

import pandas as pd
import sys
import gdal
import imp
import numpy as np


import scripts.windturbines.functions_pattern_recognition as fpr
imp.reload(fpr)
from scripts.windturbines.functions_pattern_recognition import get_param
from scripts.windturbines.functions_pattern_recognition import cop_predict
from scripts.windturbines.functions_pattern_recognition import check_image
from scripts.windturbines.functions_pattern_recognition import read_params

In [74]:
COUNTRY = "US"

train_dir = get_param(COUNTRY,"PATH_ML_IMAGES_TYPE_REGRESSION_TRAIN")
test_dir = get_param(COUNTRY,"PATH_ML_IMAGES_TYPE_REGRESSION_TEST")

#### delete directories if exist
#### create if not exist

shutil.rmtree(train_dir,ignore_errors=True)
shutil.rmtree(test_dir,ignore_errors=True)

os.makedirs(train_dir)
os.makedirs(test_dir)

src_dir_tb = get_param(COUNTRY,"PATH_RAW_IMAGES_TURBINES")


In [70]:
turbines_us = pd.read_csv(get_param("US","FILE_TURBINE_LOCATIONS"))

predictions_cs = get_param("US",
                            "PATH_RAW_IMAGES_TURBINES")

turbines_pred = pd.read_feather(predictions_cs + "all_predictions.feather")

turbines_us['prediction'] = turbines_pred['prediction']
turbines_us['filenames'] = turbines_pred['filenames']

turbines_us['id'] = np.arange(1, turbines_us.shape[0] + 1)
turbines_us.shape

turbines_us = turbines_us[turbines_pred['prediction'] > 0.99]
turbines_us = turbines_us[(~turbines_us['t_hh'].isna())]
turbines_us = turbines_us[~turbines_us['t_cap'].isna()]
turbines_us = turbines_us[~turbines_us['t_rd'].isna()]


In [72]:
turbines_us.shape

(45647, 30)

In [73]:
nmb_samples_all = turbines_us.shape[0]

#turbines_us = turbines_us[turbines_us['id'] <= nmb_samples_all]
#turbines_us.shape

In [75]:
share_train = nmb_samples_all * 0.8
share_train

turbines_us

,case_id,faa_ors,faa_asn,usgs_pr_id,t_state,t_county,t_fips,p_name,p_year,p_tnum,...,t_img_date,t_img_srce,xlong,ylat,KW,Long,Lat,prediction,filenames,id
5,3000010,NaN,NaN,16102.0,CA,Riverside County,6065,Alta Mesa 1 and 2,1988.0,115,...,11/3/2018,Digital Globe,-116.666191,33.944595,160.0,-116.666191,33.944595,1.000000,6.tif,6
23,3000075,NaN,NaN,2744.0,CA,Kern County,6029,Sky River,1990.0,342,...,11/28/2017,Digital Globe,-118.238991,35.240494,225.0,-118.238991,35.240494,0.999992,24.tif,24
25,3000078,NaN,NaN,3846.0,CA,Kern County,6029,Oasis Power Partners,2004.0,60,...,1/1/2012,NAIP,-118.296288,35.067696,1000.0,-118.296288,35.067696,1.000000,26.tif,26
29,3000086,NaN,NaN,15221.0,CA,Riverside County,6065,San Gorgonio Farms Wind Farm,2008.0,112,...,10/15/2017,Digital Globe,-116.630142,33.923340,500.0,-116.630142,33.923340,0.999998,30.tif,30
30,3000088,NaN,NaN,11050.0,CA,Alameda County,6001,Patterson Pass,1985.0,201,...,10/16/2018,Digital Globe,-121.611084,37.691296,65.0,-121.611084,37.691296,1.000000,31.tif,31
32,3000090,NaN,NaN,11091.0,CA,Alameda County,6001,Patterson Wind,1985.0,73,...,10/16/2018,Digital Globe,-121.614293,37.696745,65.0,-121.614293,37.696745,1.000000,33.tif,33
38,3000102,NaN,NaN,20799.0,IA,Winnebago County,19189,Forest City High School,1999.0,1,...,3/20/2015,Google Earth,-93.653793,43.266293,600.0,-93.653793,43.266293,0.999961,39.tif,39
41,3000110,NaN,NaN,14352.0,CA,Riverside County,6065,San Gorgonio Westwinds II LLC,1999.0,48,...,7/28/2017,Digital Globe,-116.573486,33.905193,700.0,-116.573486,33.905193,1.000000,42.tif,42
53,3000137,NaN,NaN,44088.0,TX,Crockett County,48105,Southwest Mesa,1999.0,106,...,1/9/2018,Digital Globe,-102.119492,31.078196,700.0,-102.119492,31.078196,1.000000,54.tif,54
54,3000138,NaN,NaN,16090.0,CA,Riverside County,6065,Alta Mesa 3,1995.0,42,...,11/3/2018,Digital Globe,-116.666227,33.946156,225.0,-116.666227,33.946156,1.000000,55.tif,55


In [ ]:
cnt = 0
flag = False

for i in range(0, nmb_samples_all):
    
        
        
    #print(file)
    file_s = str(turbines_us.iloc[[i]]['id'].values[0]) + '.tif'
    file_d = str(turbines_us.iloc[[i]]['id'].values[0]) + '.png'
    
    #print(file)
        
    src = os.path.join(src_dir_tb, file_s)
        
    if(cnt % 1000 == 0):
        print(cnt)
        print(src)
            
    #print(src)
        
        
        
    if(os.path.isfile(src)):
            
        dst = ""
        
        if(cnt < share_train):
           
            dst = os.path.join(train_dir, file_d)
            
        if(cnt > share_train):
                
            
            if(not flag):
                print("test files here")
                print(dst)
            
            dst = os.path.join(test_dir, file_d)
            
            if(not flag):
                print(dst)

        fpr.copyfile_to_png(src, dst)
        cnt += 1
    else:
        print("no")
        print(src)
            


0
data/aerialImages/GOOGLE/RESOLUTION19/US/raw/Turbines/6.tif
1000
data/aerialImages/GOOGLE/RESOLUTION19/US/raw/Turbines/4520.tif
2000
data/aerialImages/GOOGLE/RESOLUTION19/US/raw/Turbines/5563.tif
3000
data/aerialImages/GOOGLE/RESOLUTION19/US/raw/Turbines/6573.tif


In [77]:
print('total training turbine images ', len(os.listdir(train_dir)))
print('total testing turbine images ', len(os.listdir(test_dir)))

pd_out = turbines_us[['id','t_hh']]

pd_out['id_s'] = turbines_us['id'].map(str) + '.png'
pd_out['score'] = turbines_us['t_hh']

share_train_i = len(os.listdir(train_dir))

pd_out_train = pd_out.iloc[0: share_train_i]
pd_out_test = pd_out.iloc[((share_train_i) + 1) : pd_out.shape[0]]
                                      
pd_out_train.to_csv(train_dir + "list.csv")
pd_out_test.to_csv(test_dir + "list.csv")
pd_out

total training turbine images  36518
total testing turbine images  9129


/home/jschmidt/.conda/envs/windturbine-identification/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/jschmidt/.conda/envs/windturbine-identification/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


,id,t_hh,id_s,score
5,6,23.0,6.png,23.0
23,24,30.0,24.png,30.0
25,26,60.0,26.png,60.0
29,30,30.0,30.png,30.0
30,31,24.6,31.png,24.6
32,33,24.0,33.png,24.0
38,39,50.0,39.png,50.0
41,42,50.0,42.png,50.0
53,54,50.0,54.png,50.0
54,55,40.0,55.png,40.0


In [47]:
print(len(os.listdir(train_dir)))
print(pd_out_train.shape)
print(share_train)
print(share_train_i)



38133
(38133,)
38132.0
38133
